# Import all libraries 

In [1]:
library("data.table")
library("emmeans")
library("lmtest")
library("sandwich")
library("dplyr")
library(xtable)


Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric



Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




# Load data in 

In [ ]:
dF <- fread('zstdcat quote_centric_regression.zst')


In [4]:
nrow(dF)

[1] 6724371

In [6]:
# use a compressed version to perform regression to save RAM
df_shrunk <- select(dF, cluster_assignment,domain, party, professional_verb,unprofessional_verb,outlet_category,outlet_category_binary,relative_time,speaker_verbosity_weight,yearmonth,top_speaker,domain_proxy,verb_trf)


In [7]:
df_shrunk <- df_shrunk[df_shrunk$verb_trf != 'opine']
df_shrunk <- df_shrunk[df_shrunk$verb_trf != 'pen']
df_shrunk <- df_shrunk[df_shrunk$verb_trf != 'utter']


In [68]:
nrow(df_shrunk)

[1] 6723255

In [82]:
df_shrunk %>% group_by(outlet_category)%>%summarise(disp = mean(unprofessional_verb))

outlet_category,disp
<chr>,<dbl>
left_center_media,0.03363251
left_media,0.06674516
mainstream,0.02820376
right_center_media,0.03124684
right_media,0.05666332


In [95]:
dF <- dF[dF$verb_trf != 'opine']
dF <- dF[dF$verb_trf != 'pen']
dF <- dF[dF$verb_trf != 'utter']


In [ ]:
dF$outlet_category

# Model 1 (Equation 1), temporal trend of unobjective verb: 

In [8]:
model1=lm(unprofessional_verb ~ domain_proxy+factor(outlet_category)+ factor(outlet_category):relative_time, data = df_shrunk)


In [7]:
cl_vcov_mat1 <- vcovCL(model1,cluster = factor(df_shrunk$domain))


In [141]:
result_table_model1 <- test(emtrends(model1, specs = pairwise ~ outlet_category,var = 'relative_time',vcov = as.matrix(cl_vcov_mat1)))


In [142]:
result_table_model1$emtrends

,outlet_category,relative_time.trend,SE,df,t.ratio,p.value
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,left_center_media,5.380472e-05,1.601666e-05,6723244,3.35929713,7.814143e-04
2,left_media,-1.015518e-04,5.455714e-05,6723244,-1.86138405,6.269001e-02
3,mainstream,6.669572e-05,1.397661e-05,6723244,4.77195351,1.824514e-06
4,right_center_media,7.914832e-05,2.300137e-05,6723244,3.44102690,5.795144e-04
5,right_media,5.595200e-06,7.853086e-05,6723244,0.07124842,9.432000e-01


In [132]:
print(xtable((result_table_model1$emtrends)[,c(1,2,3,5,6)],digits=c(-5,-5,-2,-2,2,3)), include.rownames = F,booktabs=T)


% latex table generated in R 3.6.3 by xtable 1.8-4 package
% Sat Jan 14 16:00:21 2023
\begin{table}[ht]
\centering
\begin{tabular}{lrrrr}
  \toprule
outlet\_category & relative\_time.trend & SE & t.ratio & p.value \\ 
  \midrule
left\_center\_media & 5.38E-05 & 1.60E-05 & 3.36 & 0.001 \\ 
  left\_media & -1.02E-04 & 5.46E-05 & -1.86 & 0.063 \\ 
  mainstream & 6.67E-05 & 1.40E-05 & 4.77 & 0.000 \\ 
  right\_center\_media & 7.91E-05 & 2.30E-05 & 3.44 & 0.001 \\ 
  right\_media & 5.60E-06 & 7.85E-05 & 0.07 & 0.943 \\ 
   \bottomrule
\end{tabular}
\end{table}


In [157]:
emmeans_plot_1_table=as.data.table(result_table_model1$emtrends)
emmeans_plot_1_table=emmeans_plot_1_table[c(2,1,3,4,5),]
emmeans_plot_1_table$outlet_category=c('Left','Left-Center','Least-Biased', 'Right-Center','Right')
emmeans_plot_1_table$relative_time.trend=emmeans_plot_1_table$relative_time.trend*12*100
emmeans_plot_1_table$SE=emmeans_plot_1_table$SE*12*100
emmeans_plot_1_table$relative_time.trend=round(emmeans_plot_1_table$relative_time.trend,4)


In [158]:
print(xtable((emmeans_plot_1_table)[,c(1,2,3,5,6)],digits=c(-5,-2,2,2,2,3)), include.rownames = F,booktabs=T)


% latex table generated in R 3.6.3 by xtable 1.8-4 package
% Sat Jan 14 16:11:20 2023
\begin{table}[ht]
\centering
\begin{tabular}{lrrrr}
  \toprule
outlet\_category & relative\_time.trend & SE & t.ratio & p.value \\ 
  \midrule
Left & -0.12 & 0.07 & -1.86 & 0.063 \\ 
  Left-Center & 0.06 & 0.02 & 3.36 & 0.001 \\ 
  Least-Biased & 0.08 & 0.02 & 4.77 & 0.000 \\ 
  Right-Center & 0.10 & 0.03 & 3.44 & 0.001 \\ 
  Right & 0.01 & 0.09 & 0.07 & 0.943 \\ 
   \bottomrule
\end{tabular}
\end{table}


In [32]:
q2=resid(model1)

In [63]:
df_shrunk_1 <- df_shrunk[df_shrunk$unprofessional_verb ==1]


In [28]:
q1=predict(model1,df_shrunk)

In [39]:
q1[11]

11 
0.2517545

In [40]:
q2[11]

11 
-0.2517545

In [65]:
q1_1=predict(model1,df_shrunk_1)

# Model 2 (Equation 2), investigate the effect of ideology: 

In [133]:
model2=lm(unprofessional_verb ~ factor(outlet_category) + factor(party) + factor(outlet_category):factor(party), data = df_shrunk)

In [134]:
cl_vcov_mat2 <- vcovCL(model2,cluster = factor(df_shrunk$domain))


In [135]:
test(emmeans(model2, specs = pairwise ~ outlet_category:party,vcov = as.matrix(cl_vcov_mat2)))

,outlet_category,party,emmean,SE,df,t.ratio,p.value
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,left_center_media,democratic,0.02781781,0.0008240137,6723245,33.758919,8.301337e-250
2,left_media,democratic,0.05577256,0.0075660401,6723245,7.371434,1.688216e-13
3,mainstream,democratic,0.02325576,0.0005561195,6723245,41.817924,0.000000e+00
4,right_center_media,democratic,0.02900067,0.0014781548,6723245,19.619506,1.059574e-85
5,right_media,democratic,0.06723556,0.0064975440,6723245,10.347843,4.282064e-25
6,left_center_media,republican,0.03866974,0.0011038653,6723245,35.031212,7.970315e-269
7,left_media,republican,0.07466049,0.0094873609,6723245,7.869469,3.562033e-15
8,mainstream,republican,0.03208301,0.0007646488,6723245,41.957844,0.000000e+00
9,right_center_media,republican,0.03296821,0.0015627773,6723245,21.095910,8.736488e-99


In [136]:
summary(model2)


Call:
lm(formula = unprofessional_verb ~ factor(outlet_category) + 
    factor(party) + factor(outlet_category):factor(party), data = df_shrunk)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.07466 -0.03867 -0.03208 -0.02782  0.97674 

Coefficients:
                                                                    Estimate
(Intercept)                                                        0.0278178
factor(outlet_category)left_media                                  0.0279548
factor(outlet_category)mainstream                                 -0.0045620
factor(outlet_category)right_center_media                          0.0011829
factor(outlet_category)right_media                                 0.0394178
factor(party)republican                                            0.0108519
factor(outlet_category)left_media:factor(party)republican          0.0080360
factor(outlet_category)mainstream:factor(party)republican         -0.0020247
factor(outlet_category)right_center_media:

In [159]:
emmeans_plot_2=test(emmeans(model2, specs = pairwise ~ outlet_category:party,vcov = as.matrix(cl_vcov_mat2)))

In [160]:
emmeans_plot_2_table=as.data.table(emmeans_plot_2$contrasts)
emmeans_plot_2_table=emmeans_plot_2_table[c(14,5,22,29,35),]
emmeans_plot_2_table$contrast=c('Left','Left-Center','Least-Biased','Right-Center','Right')
emmeans_plot_2_table$estimate=emmeans_plot_2_table$estimate*100
emmeans_plot_2_table$SE=emmeans_plot_2_table$SE*100
emmeans_plot_2_table$estimate=round(emmeans_plot_2_table$estimate,4)

In [161]:
emmeans_plot_2_table

contrast,estimate,SE,df,t.ratio,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Left,-1.8888,0.37050089,6723245,-5.097943,1.517406e-05
Left-Center,-1.0852,0.08851599,6723245,-12.259849,0.000000e+00
Least-Biased,-0.8827,0.07767183,6723245,-11.364803,0.000000e+00
Right-Center,-0.3968,0.11070615,6723245,-3.583850,1.252434e-02
Right,1.7452,0.42029167,6723245,4.152316,1.357694e-03


In [162]:
print(xtable((emmeans_plot_2_table)[,c(1,2,3,5,6)],digits=c(-5,-2,2,2,2,3)), include.rownames = F,booktabs=T)


% latex table generated in R 3.6.3 by xtable 1.8-4 package
% Sat Jan 14 16:18:07 2023
\begin{table}[ht]
\centering
\begin{tabular}{lrrrr}
  \toprule
contrast & estimate & SE & t.ratio & p.value \\ 
  \midrule
Left & -1.89 & 0.37 & -5.10 & 0.000 \\ 
  Left-Center & -1.09 & 0.09 & -12.26 & 0.000 \\ 
  Least-Biased & -0.88 & 0.08 & -11.36 & 0.000 \\ 
  Right-Center & -0.40 & 0.11 & -3.58 & 0.013 \\ 
  Right & 1.75 & 0.42 & 4.15 & 0.001 \\ 
   \bottomrule
\end{tabular}
\end{table}


# Model 3 (Equation 3), investigate how quotative bias change over time: 

In [163]:
model3=lm(unprofessional_verb ~ domain_proxy + factor(party) + factor(outlet_category)+factor(outlet_category):factor(party):relative_time, data = df_shrunk)

In [164]:
cl_vcov_mat3 <- vcovCL(model3,cluster = factor(df_shrunk$domain))


In [165]:
test(emtrends(model3, specs = pairwise ~ outlet_category | party,var = 'relative_time',vcov = as.matrix(cl_vcov_mat3)))


,outlet_category,party,relative_time.trend,SE,df,t.ratio,p.value
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,left_center_media,democratic,-8.340343e-05,1.501678e-05,6723238,-5.5540154,2.791914e-08
2,left_media,democratic,-3.162371e-04,6.796460e-05,6723238,-4.6529682,3.271969e-06
3,mainstream,democratic,-5.338583e-05,1.565020e-05,6723238,-3.4111912,6.468009e-04
4,right_center_media,democratic,-2.119240e-05,2.097462e-05,6723238,-1.0103830,3.123119e-01
5,right_media,democratic,8.349669e-05,1.008216e-04,6723238,0.8281625,4.075785e-01
6,left_center_media,republican,1.936316e-04,2.371174e-05,6723238,8.1660626,3.186723e-16
7,left_media,republican,9.926601e-05,5.441294e-05,6723238,1.8243086,6.810547e-02
8,mainstream,republican,1.855987e-04,1.895075e-05,6723238,9.7937352,1.198290e-22
9,right_center_media,republican,1.656143e-04,2.972831e-05,6723238,5.5709295,2.533934e-08


In [30]:
summary(model3)


Call:
lm(formula = unprofessional_verb ~ domain_proxy + factor(party) + 
    factor(outlet_category) + factor(outlet_category):factor(party):relative_time, 
    data = df_shrunk)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.27656 -0.04052 -0.03108 -0.02347  1.00022 

Coefficients:
                                                                                  Estimate
(Intercept)                                                                     -7.285e-04
domain_proxy                                                                     9.901e-01
factor(party)republican                                                         -4.296e-03
factor(outlet_category)left_media                                                7.691e-03
factor(outlet_category)mainstream                                               -8.889e-04
factor(outlet_category)right_center_media                                       -7.919e-04
factor(outlet_category)right_media                              

In [166]:
emmeans_plot_3=test(emtrends(model3, specs = pairwise ~ outlet_category : party,var = 'relative_time',vcov = as.matrix(cl_vcov_mat3)))


In [168]:
emmeans_plot_3

,outlet_category,party,relative_time.trend,SE,df,t.ratio,p.value
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,left_center_media,democratic,-8.340343e-05,1.501678e-05,6723238,-5.5540154,2.791914e-08
2,left_media,democratic,-3.162371e-04,6.796460e-05,6723238,-4.6529682,3.271969e-06
3,mainstream,democratic,-5.338583e-05,1.565020e-05,6723238,-3.4111912,6.468009e-04
4,right_center_media,democratic,-2.119240e-05,2.097462e-05,6723238,-1.0103830,3.123119e-01
5,right_media,democratic,8.349669e-05,1.008216e-04,6723238,0.8281625,4.075785e-01
6,left_center_media,republican,1.936316e-04,2.371174e-05,6723238,8.1660626,3.186723e-16
7,left_media,republican,9.926601e-05,5.441294e-05,6723238,1.8243086,6.810547e-02
8,mainstream,republican,1.855987e-04,1.895075e-05,6723238,9.7937352,1.198290e-22
9,right_center_media,republican,1.656143e-04,2.972831e-05,6723238,5.5709295,2.533934e-08


In [169]:
emmeans_plot_3_table=as.data.table(emmeans_plot_3$contrasts)
emmeans_plot_3_table=emmeans_plot_3_table[c(14,5,22,29,35),]
emmeans_plot_3_table$contrast=c('Left','Left-Center','Least-Biased','Right-Center','Right')
emmeans_plot_3_table$estimate=emmeans_plot_3_table$estimate*12*100
emmeans_plot_3_table$SE=emmeans_plot_3_table$SE*12*100
emmeans_plot_3_table$estimate=round(emmeans_plot_3_table$estimate,4)


In [170]:
emmeans_plot_3_table

contrast,estimate,SE,df,t.ratio,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Left,-0.4986,0.06503376,6723238,-7.666845,8.523182e-13
Left-Center,-0.3324,0.02658697,6723238,-12.503943,0.000000e+00
Least-Biased,-0.2868,0.02353676,6723238,-12.184404,0.000000e+00
Right-Center,-0.2242,0.02865149,6723238,-7.823959,2.863265e-13
Right,0.1733,0.06397391,6723238,2.708716,1.699217e-01


In [172]:
print(xtable((emmeans_plot_3_table)[,c(1,2,3,5,6)],digits=c(-5,-2,2,2,2,3)), include.rownames = F,booktabs=T)


% latex table generated in R 3.6.3 by xtable 1.8-4 package
% Sat Jan 14 17:09:20 2023
\begin{table}[ht]
\centering
\begin{tabular}{lrrrr}
  \toprule
contrast & estimate & SE & t.ratio & p.value \\ 
  \midrule
Left & -0.50 & 0.07 & -7.67 & 0.000 \\ 
  Left-Center & -0.33 & 0.03 & -12.50 & 0.000 \\ 
  Least-Biased & -0.29 & 0.02 & -12.18 & 0.000 \\ 
  Right-Center & -0.22 & 0.03 & -7.82 & 0.000 \\ 
  Right & 0.17 & 0.06 & 2.71 & 0.170 \\ 
   \bottomrule
\end{tabular}
\end{table}


# matched analysis (for equation 2, matched analysis)

In [31]:
dF_matched <- fread('zstdcat quote_centric_regression_withmatch.zst')


In [32]:
dF_matched_selected <- select(dF_matched, outlet_lean_binary,cluster_assignment,domain, party, professional_verb,unprofessional_verb,outlet_category,outlet_category_binary,relative_time,speaker_verbosity_weight,yearmonth,top_speaker,domain_proxy,verb_trf)


In [33]:
dF_matched_selected <- dF_matched_selected[dF_matched_selected$verb_trf != 'opine']
dF_matched_selected <- dF_matched_selected[dF_matched_selected$verb_trf != 'pen']
dF_matched_selected <- dF_matched_selected[dF_matched_selected$verb_trf != 'utter']


In [34]:
model2_matched=lm(unprofessional_verb ~ factor(outlet_lean_binary) + factor(party) + factor(outlet_lean_binary):factor(party), data = dF_matched_selected)

In [35]:
cl_vcov_mat2_matched <- vcovCL(model2_matched,cluster = factor(dF_matched_selected$domain))


In [36]:
test(emmeans(model2_matched, specs = pairwise ~ outlet_lean_binary:party,vcov = as.matrix(cl_vcov_mat2_matched)))

$emmeans
 outlet_lean_binary party      emmean      SE      df t.ratio p.value
 left               democratic 0.0345 0.00106 1017119  32.533  <.0001
 right              democratic 0.0403 0.00193 1017119  20.880  <.0001
 left               republican 0.0417 0.00141 1017119  29.579  <.0001
 right              republican 0.0393 0.00149 1017119  26.445  <.0001


$contrasts
 contrast                            estimate       SE      df t.ratio p.value
 left democratic - right democratic  -0.00585 0.002203 1017119  -2.655  0.0396
 left democratic - left republican   -0.00722 0.000903 1017119  -7.997  <.0001
 left democratic - right republican  -0.00479 0.001825 1017119  -2.627  0.0427
 right democratic - left republican  -0.00137 0.002391 1017119  -0.575  0.9397
 right democratic - right republican  0.00106 0.001441 1017119   0.733  0.8839
 left republican - right republican   0.00243 0.002048 1017119   1.187  0.6352

P value adjustment: tukey method for comparing a family of 4 estimates

In [37]:
summary(model2_matched)


Call:
lm(formula = unprofessional_verb ~ factor(outlet_lean_binary) + 
    factor(party) + factor(outlet_lean_binary):factor(party), 
    data = dF_matched_selected)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.04170 -0.04170 -0.04033 -0.03927  0.96552 

Coefficients:
                                                          Estimate Std. Error
(Intercept)                                              0.0344800  0.0004127
factor(outlet_lean_binary)right                          0.0058496  0.0006381
factor(party)republican                                  0.0072240  0.0005163
factor(outlet_lean_binary)right:factor(party)republican -0.0082799  0.0008116
                                                        t value Pr(>|t|)    
(Intercept)                                              83.555   <2e-16 ***
factor(outlet_lean_binary)right                           9.167   <2e-16 ***
factor(party)republican                                  13.993   <2e-16 ***
factor(outlet_le

In [38]:
library('forestplot')

Loading required package: grid

Loading required package: magrittr

Loading required package: checkmate



# Generate Plot for Equation 1 (Figure 3)

In [39]:
emmeans_plot_1=emtrends(model1, specs = pairwise ~ outlet_category,var = 'relative_time',vcov = as.matrix(cl_vcov_mat1))
emmeans_plot_1_table=as.data.table(emmeans_plot_1$emtrends)
emmeans_plot_1_table=emmeans_plot_1_table[c(2,1,3,4,5),]
emmeans_plot_1_table$outlet_category=c('Left','Left-Center','Least-Biased', 'Right-Center','Right')
emmeans_plot_1_table$relative_time.trend=emmeans_plot_1_table$relative_time.trend*12*100
emmeans_plot_1_table$relative_time.trend=round(emmeans_plot_1_table$relative_time.trend,4)
emmeans_plot_1_table$lower.CL=emmeans_plot_1_table$lower.CL*12*100
emmeans_plot_1_table$upper.CL=emmeans_plot_1_table$upper.CL*12*100
emmeans_plot_1_table$relative_time.trend=as.character(emmeans_plot_1_table$relative_time.trend)

In [40]:
emmeans_plot_1_table

outlet_category,relative_time.trend,SE,df,lower.CL,upper.CL
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Left,-0.1219,5.455714e-05,6723244,-0.25017819,0.00645391
Left-Center,0.0646,1.601666e-05,6723244,0.02689516,0.10223616
Least-Biased,0.08,1.397661e-05,6723244,0.04716248,0.11290725
Right-Center,0.095,2.300137e-05,6723244,0.04087975,0.14907621
Right,0.0067,7.853086e-05,6723244,-0.17798699,0.19141547


In [41]:
library(repr)
# Change Jupyter plots size 
options(repr.plot.width=16, repr.plot.height=9)

In [42]:
pdf(file='model1_coeff.pdf',width = 7.1, height = 2.9,onefile=F)
emmeans_plot_1_table%>%forestplot(labeltext = c(outlet_category),
                mean = relative_time.trend,
            lower = lower.CL, 
            upper = upper.CL,
          boxsize = c(0.25/0.7227,0.25,0.25,0.25,0.25/0.7227),
           xticks = c(-0.25, -0.125,0,0.125,0.25),

                fn.ci_norm = c("fpDrawPointCI","fpDrawCircleCI","fpDrawCircleCI","fpDrawCircleCI","fpDrawPointCI"),

                col = fpColors(box = "royalblue",
                line = "darkblue",
                summary = "royalblue"),
                                  grid=TRUE,
                                  graphwidth=unit(12, "cm"),
                                  lineheight=unit(1.5, "cm"),
                lwd.zero=gpar(lwd = 2),
                                  align=c('r','l'),
                xlab=expression(paste("Percentage rate of change (", beta,")")),

               txt_gp = fpTxtGp(label =  gpar(cex=1.5),
                              ticks = gpar(cex = 1.5),
                              xlab  = gpar(cex = 1.5)))
dev.off()



png 
  2

# Generate Plot for Equation 2 (Mean) - NOT USED IN PAPER

In [43]:
emmeans_plot_2=emmeans(model2, specs = pairwise ~ outlet_category:party,vcov = as.matrix(cl_vcov_mat2))

In [44]:
emmeans_plot_2_table=as.data.table(emmeans_plot_2$emmeans)
emmeans_plot_2_table=emmeans_plot_2_table[c(2,1,3,4,5,7,6,8,9,10),]
emmeans_plot_2_table$outlet_category=c('Left','Left-Center','Least-Biased', 'Right-Center','Right','Left','Left-Center','Least-Biased', 'Right-Center','Right')
emmeans_plot_2_table$party=c(rep('Democratic',5),rep('Republican',5))
header <- tibble(outlet_category = c("Outlet Category"),
                 emmean = c("% Nonobjective"))
emmeans_plot_2_table$emmean=emmeans_plot_2_table$emmean*100
emmeans_plot_2_table$lower.CL=emmeans_plot_2_table$lower.CL*100
emmeans_plot_2_table$upper.CL=emmeans_plot_2_table$upper.CL*100

In [45]:
emmeans_plot_2_table

outlet_category,party,emmean,SE,df,lower.CL,upper.CL
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Left,Democratic,5.577256,0.0075660401,6723245,4.094340,7.060173
Left-Center,Democratic,2.781781,0.0008240137,6723245,2.620278,2.943285
Least-Biased,Democratic,2.325576,0.0005561195,6723245,2.216579,2.434574
Right-Center,Democratic,2.900067,0.0014781548,6723245,2.610354,3.189780
Right,Democratic,6.723556,0.0064975440,6723245,5.450061,7.997052
Left,Republican,7.466049,0.0094873609,6723245,5.606560,9.325538
Left-Center,Republican,3.866974,0.0011038653,6723245,3.650620,4.083328
Least-Biased,Republican,3.208301,0.0007646488,6723245,3.058433,3.358170
Right-Center,Republican,3.296821,0.0015627773,6723245,2.990522,3.603120


In [46]:
pdf(file='model2_coeff_percentage.pdf',width = 6.8, height = 3.0,onefile=F)
emmeans_plot_2_table%>%
group_by(party) %>% 
forestplot(labeltext = outlet_category, 
                mean = emmean,
            lower = lower.CL, 
            upper = upper.CL,
                boxsize = 0.25,

                fn.ci_norm = fpDrawCircleCI,
           
                col = fpColors(box = c("royalblue", "red"),
                line = c("darkblue", "red"),
                summary = c("royalblue","red")),
                        grid=TRUE,
                                  graphwidth=unit(12, "cm"),
                            lineheight=unit(1.5, "cm"),
                    align=c('r','l'),
                   xticks = c(0, 2.5, 5,7.5,10),
                xlab=expression(paste("Percentage nonobjective quotative (",gamma+eta+sigma,")")),

                lwd.zero=gpar(lwd = 2),
                 txt_gp = fpTxtGp(label =  gpar(cex=1.5),
                              ticks = gpar(cex = 1.5),
                              xlab  = gpar(cex = 1.5)))
dev.off()



png 
  2

# Generate Plot for Equation 2 (Contrast) - Figure 6

In [47]:
emmeans_plot_2=emmeans(model2, specs = pairwise ~ outlet_category:party,vcov = as.matrix(cl_vcov_mat2))

In [48]:
emmeans_plot_2_table=as.data.table(emmeans_plot_2$contrasts)
emmeans_plot_2_table=emmeans_plot_2_table[c(14,5,22,29,35),]
emmeans_plot_2_table$contrast=c('Left','Left-Center','Least-Biased','Right-Center','Right')
emmeans_plot_2_table$estimate=emmeans_plot_2_table$estimate*100
conf=confint(emmeans_plot_2$contrasts, adjust = "tukey")
conf=conf[c(14,5,22,29,35),]
emmeans_plot_2_table$lower.CL=conf$lower.CL*100
emmeans_plot_2_table$upper.CL=conf$upper.CL*100
emmeans_plot_2_table$estimate=round(emmeans_plot_2_table$estimate,4)

In [49]:
emmeans_plot_2_table

contrast,estimate,SE,df,t.ratio,p.value,lower.CL,upper.CL
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Left,-1.8888,0.0037050089,6723245,-5.097943,1.517406e-05,-3.0609400,-0.7166447
Left-Center,-1.0852,0.0008851599,6723245,-12.259849,0.000000e+00,-1.3652292,-0.8051561
Least-Biased,-0.8827,0.0007767183,6723245,-11.364803,0.000000e+00,-1.1284542,-0.6369960
Right-Center,-0.3968,0.0011070615,6723245,-3.583850,1.252434e-02,-0.7469935,-0.0465150
Right,1.7452,0.0042029167,6723245,4.152316,1.357694e-03,0.4155141,3.0748539


In [50]:
pdf(file='model2_coeff_contrast.pdf',width =8.2, height = 3.5,onefile=F)
emmeans_plot_2_table%>%forestplot(labeltext = c(contrast), 
                                  ,
                    fn.ci_norm = c("fpDrawCircleCI","fpDrawCircleCI","fpDrawCircleCI","fpDrawCircleCI","fpDrawCircleCI"),
                    boxsize = c(0.25,0.25,0.25,0.25,0.25),
                mean = estimate,
            lower = lower.CL, 
            upper = upper.CL,
                col = fpColors(box = "royalblue",
                line = "royalblue",
                summary = "royalblue"),
                                  grid=TRUE,
                                  graphwidth=unit(15, "cm"),
                                  lineheight=unit(1.5, "cm"),
                lwd.zero=gpar(lwd = 2),
                         align=c('r','l'),
                #xticks = seq(from = -.5, to = .5, by = 0.25),
                xlab = expression(atop("Democratic - Republican",paste("Percentage nonobjective quotative (",sigma,")"))),
                #xlab=expression(paste("Percentage nonobjective quotative (",gamma+eta+sigma,")")),
                #xlab= "Democratic - Republican",
                 txt_gp = fpTxtGp(label =  gpar(cex=1.5),
                              ticks = gpar(cex = 1.5),
                              xlab  = gpar(cex = 1.5)))
dev.off()



png 
  2

# Generate Plot for Equation 2 (Mean,Matched) - NOT USED IN PAPER

In [51]:
emmeans_plot_2_matched=emmeans(model2_matched, specs = pairwise ~ outlet_lean_binary:party,vcov = as.matrix(cl_vcov_mat2_matched))
emmeans_plot_2_matched_table=as.data.table(emmeans_plot_2_matched$emmeans)
emmeans_plot_2_matched_table$outlet_category=c('Left - Combined','Right - Combined','Left - Combined','Right - Combined')
emmeans_plot_2_matched_table$party=c(rep('Democratic',2),rep('Republican',2))
emmeans_plot_2_matched_table$emmean=emmeans_plot_2_matched_table$emmean*100
emmeans_plot_2_matched_table$lower.CL=emmeans_plot_2_matched_table$lower.CL*100
emmeans_plot_2_matched_table$upper.CL=emmeans_plot_2_matched_table$upper.CL*100


In [52]:
emmeans_plot_2_matched_table

outlet_lean_binary,party,emmean,SE,df,lower.CL,upper.CL,outlet_category
<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
left,Democratic,3.447996,0.001059839,1017119,3.240271,3.655721,Left - Combined
right,Democratic,4.032961,0.001931486,1017119,3.654396,4.411526,Right - Combined
left,Republican,4.170393,0.001409934,1017119,3.894050,4.446735,Left - Combined
right,Republican,3.927371,0.001485107,1017119,3.636295,4.218447,Right - Combined


In [53]:
pdf(file='model2_matched_coeff_percent_swapped.pdf',width = 6.5, height = 2.0,onefile=F)
emmeans_plot_2_matched_table%>%
group_by(outlet_category) %>% 
forestplot(labeltext = party, 
                mean = emmean,
            lower = lower.CL, 
            upper = upper.CL,
           boxsize = .25,
                            col = fpColors(box = c("royalblue", "red"),
                line = c("darkblue", "red"),
                summary = c("royalblue","red")),
                    fn.ci_norm = c(
                        rep("fpDrawCircleCI",2)),
                        grid=TRUE,
                                  graphwidth=unit(12, "cm"),
                                  lineheight=unit(2.0, "cm"),
           zero=NA,
           #lwd.zero=gpar(lwd = 2),
           align=c('r','l'),
           xticks = c(3, 3.5,4,4.5,5),
        xlab=expression(paste("Percentage nonobjective quotative (",gamma+eta+sigma,")")),

                 txt_gp = fpTxtGp(label =  gpar(cex=1.5),
                              ticks = gpar(cex = 1.5),
                              xlab  = gpar(cex = 1.5)))
dev.off()

png 
  2

# Generate Plot for Equation 2 (Contrast, Matched) - Figure 7

In [54]:
emmeans_plot_2_matched=emmeans(model2_matched, specs = pairwise ~ outlet_lean_binary:party,vcov = as.matrix(cl_vcov_mat2_matched))


In [55]:
emmeans_plot_2_matched$contrasts

 contrast                            estimate       SE      df t.ratio p.value
 left democratic - right democratic  -0.00585 0.002203 1017119  -2.655  0.0396
 left democratic - left republican   -0.00722 0.000903 1017119  -7.997  <.0001
 left democratic - right republican  -0.00479 0.001825 1017119  -2.627  0.0427
 right democratic - left republican  -0.00137 0.002391 1017119  -0.575  0.9397
 right democratic - right republican  0.00106 0.001441 1017119   0.733  0.8839
 left republican - right republican   0.00243 0.002048 1017119   1.187  0.6352

P value adjustment: tukey method for comparing a family of 4 estimates 

In [56]:
emmeans_plot_2_matched_table=as.data.table(emmeans_plot_2_matched$contrasts)
emmeans_plot_2_matched_table=emmeans_plot_2_matched_table[c(2,5),]
emmeans_plot_2_matched_table$party=c('Left (Combined)','Right (Combined)')
emmeans_plot_2_matched_table$estimate=emmeans_plot_2_matched_table$estimate*100
conf=confint(emmeans_plot_2_matched$contrasts, adjust = "tukey")
conf=conf[c(2,5),]
emmeans_plot_2_matched_table$lower.CL=conf$lower.CL*100
emmeans_plot_2_matched_table$upper.CL=conf$upper.CL*100



In [57]:
emmeans_plot_2_matched_table

contrast,estimate,SE,df,t.ratio,p.value,party,lower.CL,upper.CL
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
left democratic - left republican,-0.7223967,0.0009032911,1017119,-7.9973848,6.095124e-14,Left (Combined),-0.9544550,-0.4903383
right democratic - right republican,0.1055903,0.0014406903,1017119,0.7329145,8.838885e-01,Right (Combined),-0.2645276,0.4757082


In [58]:
pdf(file='model2_matched_coeff_contrast.pdf',width =8.5, height = 2.5,onefile=F)
emmeans_plot_2_matched_table%>%forestplot(labeltext = party, 
                                  ,
                    fn.ci_norm = c(
                        "fpDrawCircleCI","fpDrawPointCI"),
                        boxsize = c(0.25,0.25/0.7227),
                mean = estimate,
            lower = lower.CL, 
            upper = upper.CL,
                col = fpColors(box = "royalblue",
                line = "royalblue",
                summary = "royalblue"),
                                  grid=TRUE,
                                  graphwidth=unit(15, "cm"),
                                  lineheight=unit(2.0, "cm"),
                lwd.zero=gpar(lwd = 2),
                         align=c('r','l'),
                xticks = seq(from = -1.25, to = .75, by = 0.5),
                xlab = expression(atop("Democratic - Republican",paste("Percentage nonobjective quotative (",sigma,")"))),
                #xlab=expression(paste("Percentage nonobjective quotative (",gamma+eta+sigma,")")),
                #xlab= "Democratic - Republican",
                 txt_gp = fpTxtGp(label =  gpar(cex=1.5),
                              ticks = gpar(cex = 1.5),
                              xlab  = gpar(cex = 1.5)))
dev.off()




png 
  2

# Generate Plot for Equation 3 - Figure 8

In [59]:
emmeans_plot_3=(emtrends(model3, specs = pairwise ~ outlet_category:party,var = 'relative_time',vcov = as.matrix(cl_vcov_mat3)))

In [60]:
emmeans_plot_3_table=as.data.table(emmeans_plot_3$contrasts)
emmeans_plot_3_table=emmeans_plot_3_table[c(14,5,22,29,35),]
emmeans_plot_3_table$contrast=c('Left','Left-Center','Least-Biased','Right-Center','Right')
emmeans_plot_3_table$estimate=emmeans_plot_3_table$estimate*12*100
conf=confint(emmeans_plot_3$contrasts, adjust = "tukey")
conf=conf[c(14,5,22,29,35),]
emmeans_plot_3_table$lower.CL=conf$lower.CL*12*100
emmeans_plot_3_table$upper.CL=conf$upper.CL*12*100
emmeans_plot_3_table$estimate=round(emmeans_plot_3_table$estimate,4)


In [61]:
emmeans_plot_3_table

contrast,estimate,SE,df,t.ratio,p.value,lower.CL,upper.CL
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Left,-0.4986,5.419480e-05,6723238,-7.666845,8.523182e-13,-0.70434999,-0.2928575
Left-Center,-0.3324,2.215581e-05,6723238,-12.503943,0.000000e+00,-0.41655477,-0.2483292
Least-Biased,-0.2868,1.961397e-05,6723238,-12.184404,0.000000e+00,-0.36124427,-0.2123185
Right-Center,-0.2242,2.387624e-05,6723238,-7.823959,2.863265e-13,-0.31481230,-0.1335238
Right,0.1733,5.331160e-05,6723238,2.708716,1.699217e-01,-0.02910607,0.3756804


In [62]:
pdf(file='model3_coeff.pdf',width =8.2, height = 3.5,onefile=F)
emmeans_plot_3_table%>%forestplot(labeltext = c(contrast), 
                                  ,
                    fn.ci_norm = c(
                        rep("fpDrawCircleCI",4),
                                   "fpDrawPointCI"),
                    boxsize = c(rep(0.25,4),0.25/0.7227),
                mean = estimate,
            lower = lower.CL, 
            upper = upper.CL,
                col = fpColors(box = "royalblue",
                line = "royalblue",
                summary = "royalblue"),
                                  grid=TRUE,
                                  graphwidth=unit(15, "cm"),
                                  lineheight=unit(1.5, "cm"),
                lwd.zero=gpar(lwd = 2),
                         align=c('r','l'),
                xticks = seq(from = -.5, to = .5, by = 0.25),
                xlab = expression(atop("Democratic - Republican",paste("Percentage rate of change (",lambda,")"))),
                #xlab=expression(paste("Percentage nonobjective quotative (",gamma+eta+sigma,")")),
                #xlab= "Democratic - Republican",
                 txt_gp = fpTxtGp(label =  gpar(cex=1.5),
                              ticks = gpar(cex = 1.5),
                              xlab  = gpar(cex = 1.5)))
dev.off()



png 
  2